# Importing useful libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import matplotlib.image as img
from PIL import Image
import numpy as np
import random
import cv2
import os

# Setting up folder paths

In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/m10/projeto/data'

tif_images_path = f'{data_path}/tci_tifs/'
masks_images_path = f'{data_path}/masks/'

# Loading TIF images

In [ ]:
# Carregando os arquivos
tif_images_files = os.listdir(tif_images_path)
masks_images_files = os.listdir(masks_images_path)

# Listas para armazenar imagens, máscaras e metadados
images_metadatas = []
images_arrays = []
masks_images = []
masks_metadatas = []

# Carregar imagens TIFF
for file in tif_images_files:
    if file.endswith('.tif'):
        full_path = os.path.join(tif_images_path, file)
        image = cv2.imread(full_path)
        if image is not None:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_id = file.split('_')[0]  # Assume que o ID está na primeira parte do nome do arquivo

            image_metadata = {
                "id": image_id,
                "file_name": file,
                "array": image_rgb
            }

            images_metadatas.append(image_metadata)

# Carregar imagens de máscaras
for file in masks_images_files:
    if file.endswith('.png'):
        full_path = os.path.join(masks_images_path, file)
        image = cv2.imread(full_path)
        if image is not None:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            mask_id = file.split('.')[0]  # Assume que o ID é o nome do arquivo sem a extensão

            mask_metadata = {
                "id": mask_id,
                "file_name": file,
                "array": image_rgb
            }

            masks_metadatas.append(mask_metadata)

images_metadatas = sorted(images_metadatas, key=lambda x: x['id'])
masks_metadatas = sorted(masks_metadatas, key=lambda x: x['id'])

images_arrays = [metadata['array'] for metadata in images_metadatas]
masks_images = [metadata['array'] for metadata in masks_metadatas]

# Visualizing Images

In [ ]:
def plot_images(images_arrays, images_metadatas=None, num_columns=5, figsize=(15, 3)):
    """
    Plots images with their metadata as titles in a grid layout.

    Parameters:
        images_arrays (list): List of image arrays.
        images_metadatas (list, optional): List of dictionaries containing image metadata. Defaults to None.
        num_columns (int, optional): Number of columns in the grid layout. Defaults to 5.
        figsize (tuple, optional): Figure size. Defaults to (15, 3).
    """
    num_images = len(images_arrays)
    num_rows = (num_images + num_columns - 1) // num_columns

    fig, axes = plt.subplots(num_rows, num_columns, figsize=(figsize[0], figsize[1] * num_rows))
    fig.tight_layout(pad=3.0)

    axes = axes.ravel()

    for idx in range(num_images):
        ax = axes[idx]
        ax.imshow(images_arrays[idx])
        if images_metadatas is not None and 'file_name' in images_metadatas[idx]['metadata']:
            ax.set_title(f"{images_metadatas[idx]['metadata']['file_name']}\nShape {images_arrays[idx].shape}", fontsize=10)
        else:
            ax.set_title(f"Image {idx}\nShape {images_arrays[idx].shape}", fontsize=10)
        ax.grid(False)
        ax.axis('on')

    for ax in axes[num_images:]:
        ax.axis('off')

    plt.show()

In [ ]:
plot_images(images_arrays)

In [ ]:
plot_images(masks_images)

# Check if all images are the same shape

In [ ]:
def check_invalid_image_shapes(images_arrays, target_shape=(1200, 1200, 3), images_metadatas=None):
    """
    Checks each image array in the provided list for a specific shape and prints filenames or indices of those that do not match.

    Parameters:
        images_arrays (list): List of image arrays.
        target_shape (tuple): The expected shape of the images (height, width, channels).
        images_metadatas (list, optional): List of dictionaries containing image metadata. Defaults to None.

    Returns:
        bool: True if all images match the target shape, False otherwise.
    """
    all_valid = True  # Assume all images are valid until proven otherwise
    for idx, image_array in enumerate(images_arrays):
        if image_array.shape != target_shape:
            if images_metadatas and idx < len(images_metadatas) and 'file_name' in images_metadatas[idx].get('metadata', {}):
                print(f"Image {images_metadatas[idx]['metadata']['file_name']} has incorrect shape {image_array.shape}")
            else:
                print(f"Image {idx} has incorrect shape {image_array.shape}")
            all_valid = False
    return all_valid

In [ ]:
all_valid = check_invalid_image_shapes(images_arrays=images_arrays,
                                       target_shape=(1200, 1200, 3))

if all_valid:
    print("All images have the correct shape.")
else:
    print("Some images do not match the expected shape.")

# Crop images

In [ ]:
def crop_randomly(img, mask, n, new_width, new_height):
    """
    Randomly crops the given image and mask arrays into 'n' new images and masks with dimensions 'new_width' x 'new_height'.

    Parameters:
        img (numpy.ndarray): The numpy array representing the original image.
        mask (numpy.ndarray): The numpy array representing the original mask.
        n (int): The number of new images and masks to generate.
        new_width (int): The width of the new images and masks.
        new_height (int): The height of the new images and masks.

    Returns:
        tuple: A tuple containing three elements:
               - A list of numpy.ndarray representing the cropped images.
               - A list of numpy.ndarray representing the cropped masks.
               - A list of tuples containing the top-left corner coordinates of each cropped area.
    """
    original_height, original_width, _ = img.shape
    cropped_images = []
    cropped_masks = []
    crop_coordinates = []

    if new_width > original_width or new_height > original_height:
        raise ValueError("New dimensions must be smaller than or equal to the original dimensions.")

    for _ in range(n):
        # Randomly select top-left corner coordinates for cropping
        top = np.random.randint(0, original_height - new_height + 1)
        left = np.random.randint(0, original_width - new_width + 1)

        # Crop the image and mask using the same coordinates
        cropped_image = img[top:top + new_height, left:left + new_width, :]
        cropped_mask = mask[top:top + new_height, left:left + new_width]

        # Store the cropped image, mask, and coordinates
        cropped_images.append(cropped_image)
        cropped_masks.append(cropped_mask)
        crop_coordinates.append((left, top))

    return cropped_images, cropped_masks, crop_coordinates

# Visualizing cropped images

In [ ]:
def draw_rectangles_on_image(original_image, cropped_images, crop_coordinates):
    """
    Draws rectangles representing crop areas on the original image.

    Parameters:
        original_image (numpy.ndarray): The numpy array representing the original image.
        cropped_images (list): List of numpy.ndarray representing the cropped images.
        crop_coordinates (list): List of tuples containing the top-left corner coordinates of each cropped image.

    Returns:
        numpy.ndarray: The original image with rectangles drawn on it.
    """
    output_image = original_image.copy()

    for crop_coord in crop_coordinates:
        x, y = crop_coord
        cv2.rectangle(output_image, (x, y), (x + cropped_images[0].shape[1], y + cropped_images[0].shape[0]), (0, 255, 0), 2)

    return output_image

In [ ]:
cropped_images, cropped_masks, crop_coordinates = crop_randomly(images_arrays[0], masks_images[0], 5, 120, 120)

In [ ]:
plt.imshow(images_arrays[0])
plt.show()

In [ ]:
plot_images(cropped_images)

In [ ]:
plot_images(cropped_masks)

In [ ]:
plt.imshow(draw_rectangles_on_image(images_arrays[0], cropped_images, crop_coordinates))
plt.show()

# Data Augmentation - Old

In [ ]:
def data_augmentation(image, num_augmentations=3):
    """
    Applies data augmentation to a single image.

    Parameters:
        image (numpy.ndarray): The numpy array representing the original image.
        num_augmentations (int, optional): Number of augmentations to apply to the image. Defaults to 3.

    Returns:
        list: List of numpy.ndarray representing the augmented images.
    """
    augmented_images = []
    previous_transformations = set()

    while len(augmented_images) < num_augmentations:
        # Randomly choose augmentation type
        augmentation_type = random.choice(['rotate', 'translate', 'flip', 'brightness_contrast'])

        # Generate random transformation parameters
        if augmentation_type == 'rotate':
            angle = random.choice([-15, -10, -5, 5, 10, 15])  # Rotation angles in degrees
            transformation_key = ('rotate', angle)
        elif augmentation_type == 'translate':
            translation = random.choice([(15, 15), (-15, -15), (15, -15), (-15, 15), (10, 10), (-10, -10), (10, -10), (-10, 10)])  # Translations in (dx, dy) format
            transformation_key = ('translate', translation)
        elif augmentation_type == 'flip':
            flip_type = random.choice([-1, 0, 1])  # Flip horizontally, vertically, or both
            transformation_key = ('flip', flip_type)
        elif augmentation_type == 'brightness_contrast':
            alpha = random.uniform(0.3, 3.0)  # Randomly choose alpha (contrast)
            beta = random.randint(-100, 100)  # Randomly choose beta (brightness)
            transformation_key = ('brightness_contrast', alpha, beta)

        # Check if the generated transformation has already been applied
        if transformation_key in previous_transformations:
            continue  # Skip if transformation already applied

        # Apply the transformation and store the image
        if augmentation_type == 'rotate':
            augmented_image = rotate_image(image, angle)
        elif augmentation_type == 'translate':
            augmented_image = translate_image(image, translation)
        elif augmentation_type == 'flip':
            augmented_image = flip_image(image, flip_type)
        elif augmentation_type == 'brightness_contrast':
            augmented_image = adjust_brightness_contrast(image, alpha, beta)

        augmented_images.append(augmented_image)
        previous_transformations.add(transformation_key)

    return augmented_images

def rotate_image(image, angle):
    """
    Rotates the given image by the specified angle.

    Parameters:
        image (numpy.ndarray): The numpy array representing the image.
        angle (int): The rotation angle in degrees.

    Returns:
        numpy.ndarray: The rotated image.
    """
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image

def translate_image(image, translation):
    """
    Translates the given image by the specified translation (dx, dy).

    Parameters:
        image (numpy.ndarray): The numpy array representing the image.
        translation (tuple): The translation in (dx, dy) format.

    Returns:
        numpy.ndarray: The translated image.
    """
    height, width = image.shape[:2]
    translation_matrix = np.float32([[1, 0, translation[0]], [0, 1, translation[1]]])
    translated_image = cv2.warpAffine(image, translation_matrix, (width, height))
    return translated_image

def apply_random_filter(image):
    """
    Applies a random filter to the given image.

    Parameters:
        image (numpy.ndarray): The numpy array representing the image.

    Returns:
        numpy.ndarray: The image with the applied filter.
    """
    filters = [cv2.GaussianBlur, cv2.medianBlur, cv2.bilateralFilter]
    selected_filter = random.choice(filters)
    if selected_filter == cv2.GaussianBlur or selected_filter == cv2.medianBlur:
        filtered_image = selected_filter(image, (5, 5))  # Kernel size: 5x5
    else:  # selected_filter == cv2.bilateralFilter
        filtered_image = selected_filter(image, 9, 75, 75)  # d: Diameter of pixel neighborhood, sigmaColor: Filter sigma in the color space, sigmaSpace: Filter sigma in the coordinate space
    return filtered_image

def apply_random_blur(image):
    """
    Applies random blur to the given image.

    Parameters:
        image (numpy.ndarray): The numpy array representing the image.

    Returns:
        numpy.ndarray: The image with the applied blur.
    """
    blur_types = ['gaussian', 'median', 'bilateral']
    selected_blur_type = random.choice(blur_types)
    if selected_blur_type == 'gaussian':
        blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    elif selected_blur_type == 'median':
        blurred_image = cv2.medianBlur(image, 5)
    elif selected_blur_type == 'bilateral':
        blurred_image = cv2.bilateralFilter(image, 9, 75, 75)
    return blurred_image

def flip_image(image, flip_type):
  """
  Applies flip transformation to the given image.

  Parameters:
      image (numpy.ndarray): The numpy array representing the image.
      flip_type (int): Type of flip transformation.
                        0: Flip vertically
                        1: Flip horizontally
                        -1: Flip both vertically and horizontally

  Returns:
      numpy.ndarray: The image with the applied flip transformation.
  """
  return cv2.flip(image, flip_type)

def adjust_brightness_contrast(image, alpha, beta):
    """
    Applies brightness and contrast adjustment to the given image.

    Parameters:
        image (numpy.ndarray): The numpy array representing the image.
        alpha (float): Alpha value for contrast adjustment.
        beta (int): Beta value for brightness adjustment.

    Returns:
        numpy.ndarray: The image with the applied brightness and contrast adjustment.
    """
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image

In [ ]:
augmented_images = data_augmentation(cropped_images[0], 30)

In [ ]:
plt.imshow(cropped_images[0])
plt.show()

In [ ]:
plot_images(augmented_images, num_columns=5)

# Data Augmentation Adjusted

In [ ]:
class BaseImageProcess:
    """
    BaseImageProcess: A base class for image processing algorithms.

    This class provides a basic framework for implementing image processing algorithms and is intended to be subclassed.
    Subclasses should implement the `apply` method to perform specific image processing operations on an input image.
    """
    def apply(self, img):
        """
        Placeholder for applying an image processing algorithm.

        Args:
            img: The input image to process.

        Returns:
            The processed image.
        """
        pass

class Rotate(BaseImageProcess):
    def __init__(self):
        self.angle = random.choice([-15, -10, -5, 5, 10, 15])

    def apply(self, img):
        height, width = img.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), self.angle, 1)
        return cv2.warpAffine(img, rotation_matrix, (width, height))

class Translate(BaseImageProcess):
    def __init__(self):
        self.dx = random.choice([-10, -5, 0, 5, 10])
        self.dy = random.choice([-10, -5, 0, 5, 10])

    def apply(self, img):
        translation_matrix = np.float32([[1, 0, self.dx], [0, 1, self.dy]])
        height, width = img.shape[:2]
        return cv2.warpAffine(img, translation_matrix, (width, height))

class Flip(BaseImageProcess):
    def __init__(self):
        self.flip_type = random.choice([-1, 0, 1])

    def apply(self, img):
        return cv2.flip(img, self.flip_type)

class BrightnessContrast(BaseImageProcess):
    def __init__(self):
        self.alpha = random.uniform(0.5, 1.5)
        self.beta = random.randint(-50, 50)

    def apply(self, img):
        return cv2.convertScaleAbs(img, alpha=self.alpha, beta=self.beta)

class GaussianBlur(BaseImageProcess):
    def __init__(self):
        self.kernel_size = random.choice([3, 5, 7, 9, 11])

    def apply(self, img):
        return cv2.GaussianBlur(img, (self.kernel_size, self.kernel_size), 0)

class MedianBlur(BaseImageProcess):
    def __init__(self):
        self.kernel_size = random.choice([3, 5, 7, 9, 11])

    def apply(self, img):
        return cv2.medianBlur(img, self.kernel_size)

class BilateralFilter(BaseImageProcess):
    def __init__(self):
        self.d = random.choice([5, 9, 15])
        self.sigmaColor = random.choice([50, 75, 100])  # Filter sigma in the color space
        self.sigmaSpace = random.choice([50, 75, 100])  # Filter sigma in the coordinate space

    def apply(self, img):
        return cv2.bilateralFilter(img, self.d, self.sigmaColor, self.sigmaSpace)

In [ ]:
def data_augmentation(images, masks, n=3, filters=None):
    """
    Applies data augmentation to a list of images and their corresponding masks.

    Parameters:
        images (list of numpy.ndarray): The list of numpy arrays representing the original images.
        masks (list of numpy.ndarray): The list of numpy arrays representing the masks for the images.
        n (int): Number of augmentations to apply to each image.
        filters (list): List of instantiated filter classes to apply.

    Returns:
        tuple: A tuple containing two elements:
               - List of numpy.ndarray representing the original and augmented images.
               - List of numpy.ndarray representing the original and augmented masks.
    """
    all_images = []
    all_masks = []

    for image, mask in zip(images, masks):
        augmented_images = [image]  # Include the original image
        augmented_masks = [mask]    # Include the original mask
        previous_transformations = set()

        while len(augmented_images) - 1 < n:
            selected_filter = random.choice(filters)
            # Generate a unique key for the current state of the filter to avoid applying the same filter configuration twice
            transformation_key = (type(selected_filter).__name__, tuple(selected_filter.__dict__.values()))

            if transformation_key not in previous_transformations:
                augmented_image = selected_filter.apply(image)
                # Apply the filter to the mask if it's a geometric transformation
                if isinstance(selected_filter, (Rotate, Translate, Flip)):
                    augmented_mask = selected_filter.apply(mask)
                else:
                    augmented_mask = mask  # Non-geometric transformations do not modify the mask

                augmented_images.append(augmented_image)
                augmented_masks.append(augmented_mask)
                previous_transformations.add(transformation_key)

        all_images.extend(augmented_images)
        all_masks.extend(augmented_masks)

    return all_images, all_masks

In [ ]:
filters = [
    Rotate(),  # Cada chamada cria uma nova instância com um ângulo aleatório
    Translate(),
    Flip(),
    BrightnessContrast(),
    GaussianBlur(),
    MedianBlur(),
    BilateralFilter()
]

In [ ]:
augmented_images, augmented_masks = data_augmentation(cropped_images[:2], cropped_masks[:2], n=4, filters=filters)

In [ ]:
len(augmented_images), len(augmented_masks)

In [ ]:
plot_images(augmented_images)

In [ ]:
plot_images(augmented_masks)